In [1]:
from functools import reduce
from datasets import Dataset
import requests
import newspaper
import re
from collections import Counter
from urllib.parse import urlencode, urlparse, urlunparse, parse_qs
import random
import jsonlines
from tqdm import tqdm
from datasets import load_dataset, concatenate_datasets, Features
import validators
from transformers import AutoTokenizer
from datetime import datetime

/usr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**dump betöltése**

In [2]:
kmonitor_dataset = load_dataset("csv", data_files='kmdb-export-20240205.csv')['train']

kmonitor_dataset = kmonitor_dataset.filter(lambda row: row['source_url'] is not None and validators.url(row['source_url']))

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58013/58013 [00:01<00:00, 42154.48 examples/s]


In [3]:
kmonitor_dataset

Dataset({
    features: ['news_id', 'title', 'source_url', 'kmdb_url', 'newspaper', 'pub_time', 'persons', 'institutions', 'places', 'others'],
    num_rows: 57622
})

In [4]:
kmonitor_dict = {}
kmonitor_by_title = {}
for row in kmonitor_dataset:
    kmonitor_dict[row['kmdb_url']] = row
    kmonitor_by_title[row['title']] = row

**html-ek betöltése jsonl-ből**

In [2]:
htmls = {}

with jsonlines.open('kmdb_html.jsonl') as reader:
    for obj in tqdm(reader):
        htmls[obj['url']] = obj['html']

57622it [01:29, 645.49it/s] 


**Cikkek paroslása**
TODO: multiprocessing

In [8]:
def parse_article(url, html):
    if html == '':
        return {
            'title': None,
            'authors': None,
            'text': None,
            'text_cleaned': None,
            'publish_date': None,
            'meta_keywords': None,
            'meta_description': None,
        }
    article = newspaper.article(url, language='hu', input_html=html)
    article.parse()

    return {
        'title': article.title,
        'authors': article.authors,
        'text': article.text,
        'text_cleaned': article.text_cleaned,
        'publish_date': article.publish_date,
        'meta_keywords': article.meta_keywords,
        'meta_description': article.meta_description,
    }

In [10]:
dataset = dataset.map(lambda row: parse_article(row['source_url'], htmls[row['source_url']]) if row['source_url'] in htmls else parse_article(row['source_url'], ''))

Map:   0%|          | 0/57622 [00:00<?, ? examples/s]

**Így néz ki most az adathalmaz:**

In [11]:
#dataset.save_to_disk("kmdb_parsed")

Saving the dataset (0/1 shards):   0%|          | 0/57622 [00:00<?, ? examples/s]

In [5]:
from datasets import load_from_disk
dataset = load_from_disk("kmdb_parsed")

In [6]:
dataset

Dataset({
    features: ['news_id', 'title', 'source_url', 'kmdb_url', 'newspaper', 'pub_time', 'persons', 'institutions', 'places', 'others', 'authors', 'text', 'text_cleaned', 'publish_date', 'meta_keywords', 'meta_description'],
    num_rows: 57622
})

**Kiszűrjük a 300 karakternél rövidebb cikkeket**

In [7]:
dataset.filter(lambda row: row['text'])

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57622/57622 [00:02<00:00, 20778.83 examples/s]


Dataset({
    features: ['news_id', 'title', 'source_url', 'kmdb_url', 'newspaper', 'pub_time', 'persons', 'institutions', 'places', 'others', 'authors', 'text', 'text_cleaned', 'publish_date', 'meta_keywords', 'meta_description'],
    num_rows: 51082
})

In [8]:
dataset.filter(lambda row: row['text'] and len(row['text']) > 200)

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57622/57622 [00:02<00:00, 24149.02 examples/s]


Dataset({
    features: ['news_id', 'title', 'source_url', 'kmdb_url', 'newspaper', 'pub_time', 'persons', 'institutions', 'places', 'others', 'authors', 'text', 'text_cleaned', 'publish_date', 'meta_keywords', 'meta_description'],
    num_rows: 49784
})

In [9]:
dataset = dataset.filter(lambda row: row['text'] and len(row['text']) > 200)

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57622/57622 [00:02<00:00, 23602.25 examples/s]


**Néhány gyakori hiba javítása**

In [10]:
patterns = [(re.compile(p), r) for p, r in [
    (r'Fotó: (\w+\/[^\s]+ [^\s]+|MTI)', ''),
    (r'[”„]', '"'),
    (r'[…]', '...'),
    (r'\[\.\.\.\]', '...'),
    (r'^\d{4}\. [^\s]+ \d{2}\., [^\s]+, \d{2}\:\d{2} • .*$|(^Szerző: .*$)|(^Címkék: .*$)|(^Kiemelt kép: .*$)', ''),
    (r'  +', ' '),
]]

for pattern, replace in patterns:
    dataset = dataset.map(lambda row: {'meta_description':  pattern.sub(replace, row['meta_description'])})

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49784/49784 [00:09<00:00, 5197.61 examples/s]


**Megnézzük, honnan hány cikk van**

In [11]:
def get_url(source_url):
    parsed_url = urlparse(source_url)
    hostname = parsed_url.hostname.removeprefix('www.').removeprefix('m.')
    path = '/'+parsed_url.path.strip('/')+'/' if parsed_url.path.strip('/') != '' else '/'
    if '.' not in path and hostname+path not in ['hirtv.hu/belfold/', 'hirtv.hu/kulfold/', 'hirtv.hu/gazdasag/']:
        return 'https://'+hostname+path
    return 'https://'+hostname+path+'?'+parsed_url.query

dataset = dataset.map(lambda row: {'url': get_url(row['source_url'])})

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49784/49784 [00:10<00:00, 4789.33 examples/s]


In [12]:
domain_counter = Counter()

for article in dataset:
    url = urlparse(article['url'])
    domain_counter[url.netloc] += 1

sorted(domain_counter.items(), key=lambda i: i[1], reverse=True)

[('nol.hu', 5629),
 ('hvg.hu', 5459),
 ('index.hu', 5276),
 ('origo.hu', 4803),
 ('magyarnemzet.hu', 4287),
 ('24.hu', 3691),
 ('atlatszo.hu', 3089),
 ('444.hu', 2078),
 ('magyarnarancs.hu', 1734),
 ('nepszava.hu', 1140),
 ('blikk.hu', 1031),
 ('magyarhirlap.hu', 1019),
 ('vg.hu', 817),
 ('hirtv.hu', 759),
 ('napi.hu', 756),
 ('mfor.hu', 750),
 ('telex.hu', 705),
 ('g7.hu', 622),
 ('infostart.hu', 513),
 ('vs.hu', 415),
 ('magyaridok.hu', 347),
 ('hirszerzo.hu', 321),
 ('168.hu', 319),
 ('archive.figyelo.hu', 288),
 ('pestisracok.hu', 288),
 ('rtl.hu', 252),
 ('szabadeuropa.hu', 225),
 ('valasz.hu', 221),
 ('zoom.hu', 192),
 ('mno.hu', 126),
 ('valaszonline.hu', 118),
 ('168ora.hu', 115),
 ('klubradio.hu', 96),
 ('kitekinto.hu', 92),
 ('es.hu', 89),
 ('bama.hu', 83),
 ('fn.hu', 82),
 ('baon.hu', 75),
 ('hang.hu', 69),
 ('economx.hu', 67),
 ('magyarhang.org', 61),
 ('atlatszo.blog.hu', 58),
 ('vasarnapihirek.hu', 54),
 ('archivum.hvg.hu', 52),
 ('azonnali.hu', 52),
 ('kreativ.hu', 48),


**ismétlődő mezők**

In [13]:
def get_duplicate_field(field, dataset, num=1):
    field_counter = Counter()

    for article in dataset:
        field_counter[article[field]] += 1

    return sorted([(item, count) for item, count in field_counter.items() if count > num], key=lambda i: i[1])

**Ismétlődő url-ek**

In [14]:
len(get_duplicate_field('url', dataset))

476

In [15]:
uniques = set(dataset.unique("url"))

In [16]:
uniques

{'https://napi.hu/magyar_vallalatok/tobb_mint_egymilliarddal_becsult_ala_az_ugyeszseg.650501.html/?',
 'https://magyarnemzet.hu/archivum-magyarnemzet/2010/06/babolnan-is-zajlik-az-elszamoltatas/',
 'https://24.hu/fn/gazdasag/2018/02/08/spar-uzleteket-vasarol-meszaros-lorinc-erdekeltsege/',
 'https://origo.hu/nagyvilag/20131011-28-evet-kapott-detroit-volt-polgarmestere.html/?',
 'https://zoom.hu/cikk.php/?cikk=28858',
 'https://mfor.hu/cikkek/makro/micsoda-nyar-naphegyi-tetobart-nyitott-meszaros-lorinc.html/?',
 'https://magyarnemzet.hu/archivum-archivum/2004/01/andrassy-uti-muemlekeket-adott-el-az-mszp-s-onkormanyzat/',
 'https://index.hu/kultur/media/2011/09/08/botranyokkal_kisert_sorozatot_mutat_be_az_mtv/',
 'https://index.hu/belfold/2017/08/08/megfigyeles_adattarolas_adatgyujtes_bm_pinter_peterfalvi_torvenytervezet/',
 'https://magyarnemzet.hu/archivum-magyarnemzet/2008/11/negyedmillio-gyenesei-dramairojanak/',
 'https://atlatszo.hu/2018/04/20/kalandos-multu-egykori-ifjuszocialista

In [17]:
dataset

Dataset({
    features: ['news_id', 'title', 'source_url', 'kmdb_url', 'newspaper', 'pub_time', 'persons', 'institutions', 'places', 'others', 'authors', 'text', 'text_cleaned', 'publish_date', 'meta_keywords', 'meta_description', 'url'],
    num_rows: 49784
})

In [18]:
# https://github.com/huggingface/datasets/issues/2514#issuecomment-984409349
def check_uniques(example, uniques):
    """Check if current hash is still in set of unique hashes and remove if true."""
    if example["url"] in uniques:
        uniques.remove(example["url"])
        return True
    else:
        return False

In [19]:
dataset = dataset.filter(check_uniques, fn_kwargs={"uniques": uniques})

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49784/49784 [00:02<00:00, 20913.40 examples/s]


In [20]:
dataset

Dataset({
    features: ['news_id', 'title', 'source_url', 'kmdb_url', 'newspaper', 'pub_time', 'persons', 'institutions', 'places', 'others', 'authors', 'text', 'text_cleaned', 'publish_date', 'meta_keywords', 'meta_description', 'url'],
    num_rows: 49282
})

**Ismétlődő meta_description-ök**

In [21]:
get_duplicate_field('meta_description', dataset)

[('Fejezetek az orosz-magyar államadósság történetéből', 2),
 ('Előzetes letartóztatásban marad Földesi-Szabó László, az Egymásért Egy-Másért Alapítvány kuratóriumi elnöke és négy társa.',
  2),
 ('Jogerősen pert nyert a Gazdasági Versenyhivatal (GVH) három kartellező építőipari vállalkozás ellen. A Fővárosi Ítélőtábla ugyanis elutasította a Baucont Rt., az ÉPKER Kft. és a KÉSZ Kft. keresetét, amelyben a versenyhatóság döntésének hatályon kívül helyezését kérték.',
  2),
 ('Megváltoztatta az elsőfokú bíróság döntését és előzetes letartóztatásba helyezte a Fővárosi Bíróság a terézvárosi vagyonkezelő vesztegetéssel gyanúsított vezetőjét csütörtökön.',
  2),
 ('Nem regisztrált lobbista a Geuronet, ezért a lobbisták szövetsége vizsgálatot kér a a Központi Igazságügyi Hivataltól. Ha a céget esetleg megbüntetik a lobbitörvény megsértéséért, a bírság összege elenyésző lesz az elért haszonhoz képest.',
  2),
 ('Üzleti Etika', 2),
 ('A másodrendű vádlott vallomásával folytatódott kedden a Zusch

**Ismétlődő title-ök**

In [22]:
get_duplicate_field('title', dataset)

[('Riport', 2),
 ('Élet és Irodalom, 2000', 2),
 ('Földesi-Szabó előzetesben marad', 2),
 ('Vesztegetéssel gyanúsítanak egy rendőrt', 2),
 ('Gyanúsított az MSZP-s vagyonvédő', 2),
 ('Dagad a német adócsalási botrány', 2),
 ('Lemondott a BKV vezérigazgatója', 2),
 ('Vádat emelt az ügyészség Gergényi Péter ellen', 2),
 ('[origo] Üzleti Etika', 2),
 ('Őrizetbe vették Székely Zoltánt', 2),
 ('A miniszterek vallottak a pénzszórásról', 2),
 ('Politikai hírek', 2),
 ('Újbuda vezetői is érintettek?', 2),
 ('Két tanulmány húszmillióért', 2),
 ('Tiszta közélet', 2),
 ('Dataplex: néhány hét Cipruson', 2),
 ('Korrupció', 2),
 ('Ki mond igazat kvótaügyben?', 2),
 ('Vesztegetéssel gyanúsítanak egy terézvárosi alpolgármestert', 2),
 ('Házkutatás a pécsi egyetemen', 2),
 ('Kreatív Online', 2),
 ('Röviden', 2),
 ('Az ügyészség nyomoz a Wieszt-ügyben', 2),
 ('Kartellgyanús ingatlan értékbecslőknél razziázott a GVH', 2),
 ('Csalás miatt nyomoznak', 2),
 ('Korrupt rendőröket ítéltek el', 2),
 ('Szakítási 

**Ismétlődő text-ek**

In [23]:
get_duplicate_field('text', dataset)

[('Pánik lett úrrá Karácsony Gergelyen, feszült lett a viszonya a baloldali pártokkal\n\nA főpolgármester megijedt Vitézy Dávidtól, miközben szövetségesei nem bocsátják meg neki a pulykaügyet, és akár le is cserélhetik.\n\nMegéri technikumban tanulni, mutatjuk mennyit lehet keresni a tanulás mellett\n\nHa valaki reál érdeklődésű, akkor technikumba menjen, ha inkább humán beállítottságú, akkor a gimnáziumot válassza – tanácsolja Pölöskei Gáborné helyettes államtitkár.\n\nBayer Zsolt: Még egyszer, utoljára a nagyapámról\n\nEgy pillanatra képzeljük el, mit művelnének ugyanezek, ha mi, ezen az oldalon bárkiről megfogalmaztuk volna, hogy a felmenői miatt az és olyan, aki és amilyen.',
  2),
 ('Volt egyszer...\n\n2006. júl. 28-án jelent meg az első írásom a nolblogon. Majdnem tíz év. Már majdnem történelem. És nem gondoltam volna, hogy egyszer le fogom írni a kezdő mondatot. Véletlenül kerültem ide. Kerestem valamit, és egy nyereményjátékra bukkantam, amellyel szerzőket és olvasókat próbálta

In [24]:
dataset

Dataset({
    features: ['news_id', 'title', 'source_url', 'kmdb_url', 'newspaper', 'pub_time', 'persons', 'institutions', 'places', 'others', 'authors', 'text', 'text_cleaned', 'publish_date', 'meta_keywords', 'meta_description', 'url'],
    num_rows: 49282
})

**gyakori meta_description-ök törlése**

In [25]:
def replace_multiple(list, text):
    text = '\n'+text+'\n'
    for element in list:
        text = text.replace('\n'+element+'\n', '\n')
    return text.strip()

In [26]:
lines = [l for l, count in get_duplicate_field('meta_description', dataset, 3) if l != '']

dataset = dataset.map(lambda article: {'meta_description': replace_multiple(lines, article['meta_description']).strip()})

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49282/49282 [00:11<00:00, 4209.95 examples/s]


**gyakori title-ök törlése**

In [27]:
lines = [l for l, count in get_duplicate_field('title', dataset, 3) if l != '']

dataset = dataset.map(lambda article: {'title': replace_multiple(lines, article['title']).strip()})

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49282/49282 [00:09<00:00, 5211.61 examples/s]


**gyakori text-ek törlése**

In [28]:
lines = [l for l, count in get_duplicate_field('text', dataset, 3) if l != '']

dataset = dataset.map(lambda article: {'text': replace_multiple(lines, article['text']).strip()})

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49282/49282 [00:18<00:00, 2637.24 examples/s]


**üres text-ek kiszűrése**

In [29]:
dataset = dataset.filter(lambda article: article['text'].strip() != '')

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49282/49282 [00:02<00:00, 21878.11 examples/s]


In [30]:
dataset

Dataset({
    features: ['news_id', 'title', 'source_url', 'kmdb_url', 'newspaper', 'pub_time', 'persons', 'institutions', 'places', 'others', 'authors', 'text', 'text_cleaned', 'publish_date', 'meta_keywords', 'meta_description', 'url'],
    num_rows: 46950
})

**megkeressük a gyakori sorokat...**

In [31]:
common_line_counter = Counter()

for article in dataset:
    for line in article['text'].splitlines():
        if line != '':
            common_line_counter[line] += 1

common_lines = sorted([(line, count) for line, count in common_line_counter.items() if count > 5], key=lambda i: i[1])
common_lines

[('Hegyi Zoltán', 6),
 ('Szalmabálák az Eiffel-torony környékén', 6),
 ('A múltat végképp eltörölni. Mondom, eszelősök és bolsevikok.', 6),
 ('A vád', 6),
 ('Huth Gergely', 6),
 ('Jegyzetek', 6),
 ('(hirtv.hu)', 6),
 ('Putsay Gábor', 6),
 ('Szenvedi Zoltán', 6),
 ('Rosta', 6),
 ('Az árak az áfát tartalmazzák.', 6),
 ('© Fazekas István', 6),
 ('Zuschlag János', 6),
 ('Tolcsvai L. László', 6),
 ('Mire gondol?', 6),
 ('(Hír TV)', 6),
 ('Korábban kezdi kézbesíteni a Magyar Posta a februári dupla nyugdíjellátást',
  6),
 ('Ukrajna az idén még legalább félmillió embert szeretne küldeni a lassan két éve tartó háborúba. Ennek eléréséhez szigorítanának a katonai szolgálatról és mozgósításról szóló törvényen, amely a mozgósítás alá esők alsó életkorát a jelenlegi 27-ről 25 évre csökkentené, a fogyatékkal élők rokkantsági jogosultságát felülvizsgálná, illetve előirányozná, hogy a hadkötelesek immár elektronikusan is megkaphatják a katonai behívójukat. De még a külföldön élő vagy a háború miatt od

**...és töröljük őket**

In [32]:
dataset = dataset.map(lambda article: {'text': replace_multiple([line+'\n' for line, count in common_lines], article['text']+'\n').strip()})

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46950/46950 [02:31<00:00, 309.24 examples/s]


**Ha szerepel a cikk címe a cikk szövegének első sorában is, akkor töröljük a cikk első sorát**

In [33]:
len(dataset.filter(lambda article: article['text'] != '' and article['title'] in article['text'].splitlines()[0]))

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46950/46950 [00:02<00:00, 17491.81 examples/s]


988

In [34]:
def without_first_line(text):
    return '\n'.join(text.splitlines()[1:]).strip()

In [35]:
dataset = dataset.map(lambda article: {'text': without_first_line(article['text'])}
                      if article['text'] != '' and article['title'] in article['text'].splitlines()[0] else {})

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46950/46950 [00:13<00:00, 3576.33 examples/s]


**Most pedig azt nézzük, hogy a meta_description-el kezdődik a text**

In [36]:
len(dataset.filter(lambda article: article['text'] != '' and article['meta_description'] in article['text'].splitlines()[0] and article['meta_description'] != ''))

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46950/46950 [00:02<00:00, 16134.92 examples/s]


6504

In [37]:
dataset = dataset.map(lambda article: {'text': without_first_line(article['text'])}
                      if article['text'] != '' and article['meta_description'] in article['text'].splitlines()[0] else {})

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46950/46950 [00:12<00:00, 3844.40 examples/s]


**Hány cikkhez nincs szöveg, lead vagy cím?**

In [38]:
len(dataset.filter(lambda article: article['text'] == ''))

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46950/46950 [00:02<00:00, 21131.91 examples/s]


886

In [39]:
len(dataset.filter(lambda article: article['meta_description'] == ''))

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46950/46950 [00:02<00:00, 22435.07 examples/s]


2214

In [40]:
len(dataset.filter(lambda article: len(article['title']) < 5))

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46950/46950 [00:02<00:00, 21473.16 examples/s]


793

In [41]:
len(dataset.filter(lambda article: article['title'] == article['meta_description']))

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46950/46950 [00:02<00:00, 22802.84 examples/s]


1651

In [42]:
set(dataset.filter(lambda article: len(article['meta_description']) < 5)['meta_description'])

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46950/46950 [00:02<00:00, 20909.96 examples/s]


{'', '-', '🙏'}

**Ha nincs meta_description egy cikkhez, megpróbáljuk a törzs első sorát használni**

In [43]:
dataset = dataset.map(lambda article: {'text': without_first_line(article['text']), 'meta_description': article['text'].splitlines()[0]}
                      if len(article['meta_description']) < 15 and article['text'] != '' else {})

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46950/46950 [00:11<00:00, 4124.80 examples/s]


**Maradék hiányos elemek kiszűrése**

In [44]:
dataset = dataset.filter(lambda article: not (
               len(article['meta_description']) < 5 or
               article['title'] == article['meta_description'] or
               len(article['title']) < 5 or
               article['meta_description'] == '' or
               article['text'] == ''
            ))

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46950/46950 [00:02<00:00, 20690.71 examples/s]


**kódolás javítása**

In [45]:
encoding_fix = {
    'Å\x91': 'ő',
    'Ã¡': 'á',
    'Ã³': 'ó',
    'Ã\xad': 'í',
    'Ãº': 'ú',
    'Ã©': 'é',
    'Ã¼': 'ü',
    'Å±': 'ű',
    'Ã¶': 'ö',
    'Ã\x81': 'Á',
    'Ã\x9a': 'Ú',
    'Ã\x96': 'Ö',
    'Ã\x9c': 'Ü',
    'Ã\x89': 'É',
    'Ã\x8d': 'Í',
    'Ã\x93': 'Ó',
    'Å\x90': 'Ő',
    'Ä\x83': 'ă',
    'â\x80\x93': '-',
    'â\x80\x9e': '"',
    'â\x80\x9d': '"',
    'â\x80\x99': '"',
    'â\x80\x99': '"',
    'â\x80¢': '-',
    '\x96': '-',
    '\x94': '"',
    '\x84': '"',
    'ô': 'ő',
    'Ô': 'Ő',
    'õ': 'ő',
    'Õ': 'Ő',
    'û': 'ű',
    'Û': 'Ű',
}

def replace_dict(text, dict):
    for key in dict:
        text = text.replace(key, dict[key])
    return text

In [46]:
dataset = dataset.map(lambda article: {'text': replace_dict(article['text'], encoding_fix)})
dataset = dataset.map(lambda article: {'meta_description': replace_dict(article['meta_description'], encoding_fix)})
dataset = dataset.map(lambda article: {'title': replace_dict(article['title'], encoding_fix)})

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44427/44427 [00:10<00:00, 4263.83 examples/s]


**tokenizálással megnézzük a cikkek hosszát**

In [47]:
tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")

In [48]:
dataset = dataset.map(lambda article: {'length': len(tokenizer(article['text'])['input_ids'])})

Map:   0%|                                                                                                                             | 18/44427 [00:00<03:37, 204.55 examples/s]


KeyboardInterrupt: 

In [ ]:
dataset

**egyesítjük a régi gyűjtést a jelenlegivel**

In [49]:
base_dataset = load_dataset("boapps/kmdb_base", revision='096edd5cba88e31fe1aa9b8a5f48cfbf7e2c3dc9')

In [51]:
dataset = dataset.map(lambda article: {'kmonitor_title': kmonitor_dict[article['kmdb_url']]['title'] })

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44427/44427 [00:09<00:00, 4730.92 examples/s]


In [52]:
ourls = set(base_dataset['train'].unique('kmonitor_title')+base_dataset['validation'].unique('kmonitor_title')+base_dataset['test'].unique('kmonitor_title'))
nurls = set(dataset.unique('kmonitor_title'))

In [53]:
diff = ourls.difference(nurls)

In [54]:
base_dataset = base_dataset.filter(lambda article: article['kmonitor_title'] in diff)

Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1710/1710 [00:00<00:00, 15848.72 examples/s]


In [55]:
base_dataset = base_dataset.rename_column('keywords', 'meta_keywords').rename_column('description', 'meta_description').rename_column('date', 'publish_date').remove_columns('kmonitor_collections').rename_column('kmonitor_tags', 'others').rename_column('kmonitor_institutes', 'institutions').rename_column('kmonitor_people', 'persons').rename_column('kmonitor_places', 'places')

In [56]:
base_dataset = base_dataset.filter(lambda article: article['kmonitor_title'] in kmonitor_by_title)

Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 168/168 [00:00<00:00, 9631.40 examples/s]


In [57]:
base_dataset=base_dataset.map(lambda article: {'news_id': kmonitor_by_title[article['kmonitor_title']]['news_id'],
                                 'source_url': kmonitor_by_title[article['kmonitor_title']]['source_url'],
                                 'kmdb_url': kmonitor_by_title[article['kmonitor_title']]['kmdb_url'],
                                 'newspaper': kmonitor_by_title[article['kmonitor_title']]['newspaper'],
                                 'pub_time': kmonitor_by_title[article['kmonitor_title']]['pub_time'],
                                 'others': kmonitor_by_title[article['kmonitor_title']]['others'],
                                 'institutions': kmonitor_by_title[article['kmonitor_title']]['institutions'],
                                 'places': kmonitor_by_title[article['kmonitor_title']]['places'],
                                 'persons': kmonitor_by_title[article['kmonitor_title']]['persons'],
                                 'authors': [],
                                 })

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 147/147 [00:00<00:00, 3086.32 examples/s]


In [59]:
dataset = dataset.remove_columns(['text_cleaned', 'length'])

In [60]:
dataset = dataset.map(lambda article: {'publish_date2': article['publish_date'].strftime("%Y-%m-%d %H:%M:%S%z") if article['publish_date'] else None}).remove_columns('publish_date').rename_column('publish_date2', 'publish_date')

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44427/44427 [00:09<00:00, 4733.66 examples/s]


In [61]:
concatenate_datasets([dataset, base_dataset['train'], base_dataset['validation'], base_dataset['test']]).save_to_disk('kmdb')

/usr/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)
Saving the dataset (1/1 shards): 100%|███████████████████████████████████████████████████████████████████████████████████████████| 47024/47024 [00:00<00:00, 151973.91 examples/s]


In [62]:
dataset = concatenate_datasets([dataset, base_dataset['train'], base_dataset['validation'], base_dataset['test']])

In [63]:
dataset

Dataset({
    features: ['news_id', 'title', 'source_url', 'kmdb_url', 'newspaper', 'pub_time', 'persons', 'institutions', 'places', 'others', 'authors', 'text', 'meta_keywords', 'meta_description', 'url', 'kmonitor_title', 'publish_date', 'kmonitor_description'],
    num_rows: 47024
})

In [64]:
uniques = set(dataset.unique("source_url"))
def check_uniques(example, uniques):
    """Check if current hash is still in set of unique hashes and remove if true."""
    if example["source_url"] in uniques:
        uniques.remove(example["source_url"])
        return True
    else:
        return False
dataset = dataset.filter(check_uniques, fn_kwargs={"uniques": uniques})

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47024/47024 [00:01<00:00, 26511.45 examples/s]


In [65]:
dataset

Dataset({
    features: ['news_id', 'title', 'source_url', 'kmdb_url', 'newspaper', 'pub_time', 'persons', 'institutions', 'places', 'others', 'authors', 'text', 'meta_keywords', 'meta_description', 'url', 'kmonitor_title', 'publish_date', 'kmonitor_description'],
    num_rows: 46961
})

In [66]:
dataset = dataset.map(lambda article: {'kmonitor_places': article['places'].split(', ') if article['places'] != '\\N' else []})
dataset = dataset.map(lambda article: {'kmonitor_persons': article['persons'].split(', ') if article['persons'] != '\\N' else []})
dataset = dataset.map(lambda article: {'kmonitor_others': article['others'].split(', ') if article['others'] != '\\N' else []})
dataset = dataset.map(lambda article: {'kmonitor_institutions': article['institutions'].split(', ') if article['institutions'] != '\\N' else []})
dataset = dataset.remove_columns(['places', 'persons', 'others', 'institutions'])

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46961/46961 [00:11<00:00, 3981.08 examples/s]


In [67]:
uniques = set(dataset.unique("kmdb_url"))
def check_uniques(example, uniques):
    """Check if current hash is still in set of unique hashes and remove if true."""
    if example["kmdb_url"] in uniques:
        uniques.remove(example["kmdb_url"])
        return True
    else:
        return False
dataset = dataset.filter(check_uniques, fn_kwargs={"uniques": uniques})

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46961/46961 [00:02<00:00, 21205.85 examples/s]


In [79]:
dataset[0]

{'news_id': 1,
 'title': 'szavazatok a balliberálisoknak – eladták a Kopaszi',
 'source_url': 'https://index.hu/belfold/budapest/leisznduha62/',
 'kmdb_url': 'https://adatbazis.k-monitor.hu/hirek/magyar-hirek/mdf-voksok-a-balliberalisoknak-a-eladtak-a-kopaszi-gatat',
 'newspaper': 'Index',
 'pub_time': '2006-12-13 00:00:00',
 'authors': ['Sz. I. M'],
 'text': 'A XI. kerületi önkormányzat több mint másfélmilliárd forintért eladta utolsó, 25 százalékos üzletrészét az Öböl XI. Kft.-ben, amely a lágymányosi Kopaszi-gát hasznosítására alakult. Ezzel a kerület gyakorlatilag kiszállt abból a cégből, amelyet Leisztinger Tamás nagyvállalkozóval közösen alapított, és amely a gáton szórakozónegyedet és üdülőparkot épít ki.\n\nAz üzletnek ad egy vajszínű árnyalatot, hogy Újbuda, azaz a XI. kerület vezetése hemzseg a fővárosi szinten meghatározó pártpolitikusoktól: Molnár Gyula kerületi polgármester az MSZP fővárosi elnöke, Lakos Imre alpolgármester pedig az SZDSZ-ben tölt be hasonló funkciót, ehhe

In [75]:
dataset

Dataset({
    features: ['news_id', 'title', 'source_url', 'kmdb_url', 'newspaper', 'pub_time', 'authors', 'text', 'meta_keywords', 'meta_description', 'url', 'kmonitor_title', 'publish_date', 'kmonitor_description', 'kmonitor_places', 'kmonitor_persons', 'kmonitor_others', 'kmonitor_institutions'],
    num_rows: 46914
})

In [76]:
dataset.save_to_disk('kmdb')

Saving the dataset (1/1 shards): 100%|████████████████████████████████████████████████████████████████████████████████████████████| 46914/46914 [00:02<00:00, 21502.67 examples/s]


In [78]:
dataset.push_to_hub('boapps/kmdb_base')

Deleting unused files from dataset repository: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.66it/s]
